# Построение предсказания на основе агрегирации ответов моделей

In [1]:
import os
import re
import pickle
import json
import numpy as np
import pandas as pd
from hack_lap.utils.evaluate import precision_recall

In [2]:
import matplotlib
# matplotlib.use('Qt5Agg')
# %matplotlib qt5
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
DIR_DATA = os.path.join('..', 'data')
DIR_PREDICT = os.path.join(DIR_DATA, 'predict') 
DIR_MODEL = os.path.join(DIR_DATA, 'model')
files = [n for n in os.listdir(DIR_MODEL) if n.endswith('.pkl') and 'Nepoch' in n]

ps = re.compile(f'seed-(\d+)', re.I)
seeds = sorted([int(g.group(1)) for f in files for g in ps.finditer(f)])
len(files)

24

In [4]:
predict = []
for f in files:
    with open(os.path.join(DIR_MODEL, f), 'rb') as fp:
        data = pickle.load(fp)
    yt_dev, yp_dev, yp_test = data['yt_dev'], data['yp_dev'], data['yp_test']

    yp_dev = np.mean(yp_dev, axis=1).ravel()
    rp0, rp1, th = precision_recall(yt_dev, yp_dev)
    f1 = 2 * rp1[0] * rp1[1] / (rp1[0] + rp1[1] + 1e-6)
    ii = np.argmax(f1)
    f1 = f1[ii]
    th = th[ii]

    yp_test = np.mean(yp_test, axis=1)
    # yp_test = np.median(yp_test, axis=1)
    yp_test = (yp_test > th).astype(int).reshape(-1, 1)
    predict.append(yp_test)

In [22]:
submission = np.concatenate(predict, axis=1)
print(submission.shape[1])
# th = submission.shape[1] // 2
th = 7
print(th)
submission = np.sum(submission, axis=1)
submission = (submission > th).astype(int)

24
7


In [23]:
df = pd.read_csv(os.path.join(DIR_DATA, 'test.csv'))
df['Active'] = submission

seed_hash = hash(''.join(map(str, seeds)))
name = f'mdl-L4-H32-Pmean-NF-BF-#seeds-{len(seeds)}_hash-{seed_hash}_mean_th-{th}'

df.to_csv(os.path.join(DIR_PREDICT, name + '.csv'), index=False)

with open(os.path.join(DIR_PREDICT, name + '.json'), 'w') as fp:
    json.dump({'seeds': seeds}, fp, indent=2)